In [2]:
#import dataset

import tensorflow as tf 
import tensorflow_datasets as tfds

In [5]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True, download=False)

AssertionError: Dataset imdb_reviews: could not find data in C:\Users\ASUS\tensorflow_datasets. Please make sure to call dataset_builder.download_and_prepare(), or pass download=True to tfds.load() before trying to access the tf.data.Dataset object.